## 라이브러리 import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 데이터 (최종 선택된 특징데이터) 불러오기

In [ ]:
FeatureSelected = pd.read_csv('./ProcessedData/FeatureSelected', sep=',', header=None)
FeatureSelected.shape

## K값에 맞춰 Fold별 학습/검증 데이터 분할

K값(Fold개수), 데이터 개수(정상/고장 각각), 특징 수(=데이터 차원) 정의하여 '1개 Fold의 (검증)데이터 개수' 계산

In [ ]:
NoOfData   = int(FeatureSelected.shape[1]/2)   # 데이터 개수 (정상/고장 각각)
Fold       = 5                   # Fold 개수 선정 : 데이터 개수의 약수여야 함.

FeatNo     = int(FeatureSelected.shape[0])  # 데이터 특징 수 (=데이터 차원)
FoldDataNo = int(NoOfData/Fold)            # 1개 Fold 당 (검증)데이터 개수

print(FeatNo)
print(FoldDataNo)

Fold별 데이터 분할 용이하도록 데이터 Reshape

In [ ]:
NormalSet   = np.array(FeatureSelected.iloc[: , :NoOfData])
AbnormalSet = np.array(FeatureSelected.iloc[: , NoOfData:])
FeatureSelected_Reshaped = pd.DataFrame(np.concatenate([NormalSet , AbnormalSet] , axis=0))
FeatureSelected_Reshaped.shape

Fold별 데이터 분할 (정상/고장 데이터 동일하게 분포하도록)

In [ ]:
# Validation Data set
for i in range(Fold):
    
    temp_Valid_Normal   = FeatureSelected_Reshaped.iloc[:FeatNo , FoldDataNo*i : FoldDataNo*(i+1)]
    temp_Valid_Abnormal = FeatureSelected_Reshaped.iloc[FeatNo: , FoldDataNo*i : FoldDataNo*(i+1)]
    temp_Valid = pd.DataFrame(np.transpose(np.concatenate([temp_Valid_Normal, temp_Valid_Abnormal] , axis=1)))
    
    s = 'Validation_Fold%d = temp_Valid'%(i+1)
    exec(s)

    
# Training Data set
for i in range(Fold):
    
    temp_Train_Front = FeatureSelected_Reshaped.iloc[:,:FoldDataNo*i]
    temp_Train_Back  = FeatureSelected_Reshaped.iloc[:,FoldDataNo*(i+1):]
    temp_Train_Total = np.concatenate([temp_Train_Front , temp_Train_Back] , axis=1)
    temp_Train_Final = pd.DataFrame(np.transpose(np.concatenate([temp_Train_Total[:FeatNo,:],temp_Train_Total[FeatNo:,:]] , axis=1)))
    
    s ='Training_Fold%d  = temp_Train_Final'%(i+1)
    exec(s)


데이터 분할 결과(shape) 확인

In [ ]:
Validation_Fold1.shape

In [ ]:
Training_Fold1.shape

## Fold 별 학습/검증 데이터 개수에 맞춰 Labeling

Fold별 학습/검증 데이터 개수 확인

In [ ]:
NoOfLabel_Train = int(Training_Fold1.shape[0]/2)
NoOfLabel_Valid = int(Validation_Fold1.shape[0]/2)

print(NoOfLabel_Train)
print(NoOfLabel_Valid)

#### (1) KNN & SVM 레이블 - 정상: 0 // 고장: 1

In [ ]:
TrainingFold_Label   = np.zeros(2*NoOfLabel_Train , dtype=int)
ValidationFold_Label = np.zeros(2*NoOfLabel_Valid , dtype=int)

# 고장데이터(학습용) Label 값 = 1
TrainingFold_Label[NoOfLabel_Train:] = 1

# 고장데이터(검증용) Label 값 = 1
ValidationFold_Label[NoOfLabel_Valid:] = 1

TrainingFold_Label   = pd.Series(TrainingFold_Label)
ValidationFold_Label = pd.Series(ValidationFold_Label)

#TrainingFold_Label
ValidationFold_Label

#### (2) ANN 레이블 (One-hot encoding) - 정상: [1, 0] // 고장: [0, 1]

In [ ]:
TrainingFold_Label_forANN   = np.zeros((NoOfLabel_Train*2,2) , dtype=int)
ValidationFold_Label_forANN = np.zeros((NoOfLabel_Valid*2,2) , dtype=int)

TrainingFold_Label_forANN[:NoOfLabel_Train , 0] = 1    # 정상(학습)데이터 레이블 : [1,0]
TrainingFold_Label_forANN[NoOfLabel_Train: , 1] = 1    # 고장(학습)데이터 레이블 : [0,1]

ValidationFold_Label_forANN[:NoOfLabel_Valid , 0] = 1  # 정상(검증)데이터 레이블 : [1,0]
ValidationFold_Label_forANN[NoOfLabel_Valid: , 1] = 1  # 고장(검증)데이터 레이블 : [0,1]

TrainingFold_Label_forANN   = pd.DataFrame(TrainingFold_Label_forANN)
ValidationFold_Label_forANN = pd.DataFrame(ValidationFold_Label_forANN)

TrainingFold_Label_forANN
#ValidationFold_Label_forANN

## k-fold 데이터 및 레이블 저장

k-fold 데이터 (Training & Validation) 저장

In [ ]:
for i in range(Fold):
    
    path1 = './K_FoldData/Training_Fold%d'  %(i+1)
    path2 = './K_FoldData/Validation_Fold%d'%(i+1)
    
    c1 = 'Training_Fold%d.to_csv(  path1, sep = ",", header = None, index = None)'%(i+1)
    c2 = 'Validation_Fold%d.to_csv(path2, sep = ",", header = None, index = None)'%(i+1)
    exec(c1)
    exec(c2)
    

레이블 (Training & Validation) 저장

In [ ]:
TrainingFold_Label.to_csv(  './K_FoldData/TrainingFold_Label', header = None, index = None)
ValidationFold_Label.to_csv('./K_FoldData/ValidationFold_Label', header = None, index = None)

TrainingFold_Label_forANN.to_csv(  './K_FoldData/TrainingFold_Label_forANN', sep = ",", header = None, index = None)
ValidationFold_Label_forANN.to_csv('./K_FoldData/ValidationFold_Label_forANN', sep = ",", header = None, index = None)

## 최종 AI모델 학습용 전체 데이터 및 레이블 저장

전체 데이터 저장 (행렬 전치)

In [ ]:
Training_All = np.transpose(FeatureSelected)

Training_All.shape

#### 전체 데이터 레이블 (1) SVM 및 KNN 맞춤형

In [ ]:
Training_All_Label = np.zeros(NoOfData*2)

Training_All_Label[NoOfData:] = 1    # 고장데이터(학습용) Label 값 = 1
Training_All_Label = pd.Series(Training_All_Label)

Training_All_Label.shape

#### 전체 데이터 레이블 (2) ANN 맞춤형 (One-hot encoding)

In [ ]:
Training_All_Label_forANN = np.zeros((NoOfData*2,2))

Training_All_Label_forANN[:NoOfData , 0] = 1
Training_All_Label_forANN[NoOfData: , 1] = 1
Training_All_Label_forANN = pd.DataFrame(Training_All_Label_forANN)

Training_All_Label_forANN.shape

전체 데이터 & 레이블 저장

In [ ]:
Training_All.to_csv('./K_FoldData/Training_All', sep = ",", header = None, index = None)
Training_All_Label.to_csv('./K_FoldData/Training_All_Label', sep = ",", header = None, index = None)
Training_All_Label_forANN.to_csv('./K_FoldData/Training_All_Label_forANN', sep = ",", header = None, index = None)

# [실습 과제 3]

## 6개 Fold로 K-Fold 데이터 생성(분할)
#### >>>>>> 저장된 6개 Fold 학습/검증 데이터  압축하여 zip파일 제출
#### >>>>>> 데이터 이름 : ST(수강생 번호)_HW3.zip  (예시 : 'ST000_HW3.zip'  //  'ST00_HW3.zip'  //  'ST0_HW3.zip')
#### >>>>>> 데이터 이름 중 'ST' , 'HW' 등 영어는 모두 대문자

### ****** 필독 !! 실습과제 주의사항 ******

- 각자의 "수강생 번호" 확인 (아이캠퍼스 공지) 
- 제출하는 실습과제 파일에 "수강생 번호"를 기준으로 작성 (이름, 학번 등 작성X)
- 각 실습과제에 대한 구체적인 파일 이름은 매번 개별 안내 
  (수강생 번호 123번 학생 과제파일 예시 : 'ST123_HW2_1.csv' )
- 과제 파일이름 양식 지키지 않을 시 감점!